In [6]:
import transformers, huggingface_hub, nltk, torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tarikrashada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Model

In [13]:
model_ckpt = 'facebook/bart-large-mnli'

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

def load_model(task,model_ckpt,device):
    pipe = transformers.pipeline(task=task,
                                model=model_ckpt,
                                device=device)
    return pipe

print(device)

mps


In [14]:
theme_classifier = load_model('zero-shot-classification',model_ckpt,device)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
theme_list = ['coming-of-age','journey','self development','']

## Process Dataset

In [44]:
subtitle_path = '../data/subtitles.txt'
import pandas as pd

def datasetProcessing(path):
    dialogue = []
    with open(path,'r') as file:
        lines = file.readlines()
        lines = lines[1:]
        
    for line in lines: 
        dialogue.append(line.split(',')[2])
    
    for i in range(len(dialogue)):
        dialogue[i] = dialogue[i].replace('"','')
        
    script = " ".join(dialogue)
    
    return script

script = datasetProcessing(subtitle_path)

In [59]:
script

"Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness!\n We're doomed!\n There'll be no escape for the Princess this time.\n What's that?\n I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...\n Hurry up!  Come with me!  What are you waiting for?!  Get in gear!\n Artoo! Artoo-Detoo At last!  Where have you been?\n They're heading in this direction. What are we going to do?  We'll be sent to the spice mines of Kessel or smashed into who knows what!\n Wait a minute The Death Star plans are not in the main computer.\n Where are those transmissions you intercepted?\n We intercepted no transmissions. Aaah...  This is a consular ship. Were on a diplomatic mission.\n If this is a consular ship... where is the Ambassador?\n Commander There she is! Set for stun!\n She'll be all right. Inform Lord Vader we have a prisoner.\n Hey Don't call me a mindless philosopher Secret mission?  What plans?  

In [57]:
import numpy as np
def getThemeOutputs(script,theme_list,pipeline):
    
    script_sentences = nltk.sent_tokenize(script)
    batch_size = 20
    script_batches = []
    
    for index in range(0, len(script_sentences), batch_size):
        sentences = " ".join(script_sentences[index:index + batch_size])
        script_batches.append(sentences)
        
    themes = {}
    outputs = pipeline(script_batches,theme_list,multi_label=True)
    for output in outputs:
        for label, score in zip(output['labels'],output['scores']):
            if label not in themes:
                themes[label] = []
            themes[label].append(score)
    
    themes = {key : np.mean(np.array(value)) for key,value in themes.items()}
    return themes
            

In [58]:
getThemeOutputs(script,['force','self development','journey','peace','adventure','hope','despair'],theme_classifier)

{'journey': 0.7449965691707265,
 'force': 0.820058349901297,
 'despair': 0.2648075387362156,
 'adventure': 0.7728917087761935,
 'self development': 0.6455037878958259,
 'hope': 0.4343321297123412,
 'peace': 0.16673999246027446}

In [60]:
score_dict = {'journey': 0.7449965691707265,
    'force': 0.820058349901297,
    'despair': 0.2648075387362156,
    'adventure': 0.7728917087761935,
    'self development': 0.6455037878958259,
    'hope': 0.4343321297123412,
    'peace': 0.16673999246027446}

In [72]:
pd.DataFrame.from_dict(data=score_dict,orient='index').transpose()

,journey,force,despair,adventure,self development,hope,peace
0,0.744997,0.820058,0.264808,0.772892,0.645504,0.434332,0.16674
